# Reinforcement Learning - TensorFlow 2

### Import Pacakages

In [4]:
import random
import gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import os

### Hyperparameters

In [5]:
env = gym.make('CartPole-v0') # specify the environment
state_size = env.observation_space.shape[0] # get the state from env
action_size = env.action_space.n # get action size
batch_size = 32 # mini batch size
n_episodes = 1000 # number of episodes 
output_dir = 'model_output/cartpole/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### DQN

In [7]:
class DQNAgent:
    def __init__(self, state_size, action_size):
           self.state_size = state_size
           self.action_size = action_size
           self.memory = deque(maxlen=2000) # storing memories that can be replayed for training DQN
           self.gamma = 0.95 #discount future reward
           self.epsilon = 1.0 # exploration rate. how often we allow the model to explore, relative to exploiting 
           self.epsilon_decay = 0.995 # as the agent gains experience, we slowy decy(reduce) the exploration
           self.epsilon_min = 0.01 # floor on how low can the exploration rate decay
           self.learning_rate = 0.001 
           self.model = self._build_model()

    def _build_model(self):
           model = Sequential()
           model.add(Dense(32, activation='relu',
                           input_dim=self.state_size))
           model.add(Dense(32, activation='relu'))
           model.add(Dense(self.action_size, activation='linear'))
           model.compile(loss='mse',
                         optimizer=Adam(lr=self.learning_rate))
           return model

    # append the memory deque at every state
    def remember(self, state, action, reward, next_state, done):
           self.memory.append((state, action,
                               reward, next_state, done))
    # train by replaying memories 
    def train(self, batch_size):
           minibatch = random.sample(self.memory, batch_size)
           for state, action, reward, next_state, done in minibatch:
               target = reward # if done
               if not done:

                   target = (reward +
                             self.gamma *
                             np.amax(self.model.predict(next_state)[0]))
               target_f = self.model.predict(state)
               target_f[0][action] = target
               self.model.fit(state, target_f, epochs=1, verbose=0)
           if self.epsilon >  self.epsilon_min:
               self.epsilon *= self.epsilon_decay

    # select action 
    def act(self, state):
           if np.random.rand() <= self.epsilon:
               return random.randrange(self.action_size)
           act_values = self.model.predict(state)
           return np.argmax(act_values[0])

    def save(self, name):
           self.model.save_weights(name)

    def load(self, name):
           self.model.load_weights(name)

In [8]:
agent = DQNAgent(state_size, action_size)

### Training

In [10]:
for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    done = False
    time = 0
    while not done:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
           print("episode: {}/{}, score: {}, e: {:.2}"
                 .format(e, n_episodes-1, time, agent.epsilon))
        time += 1
        if len(agent.memory) > batch_size:
            agent.train(batch_size)
        if e % 50 == 0:
            agent.save(output_dir + "weights_"
                   + '{:04d}'.format(e) + ".hdf5")

episode: 0/999, score: 42, e: 0.95
episode: 1/999, score: 20, e: 0.86
episode: 2/999, score: 15, e: 0.79
episode: 3/999, score: 11, e: 0.74
episode: 4/999, score: 21, e: 0.67
episode: 5/999, score: 21, e: 0.6
episode: 6/999, score: 10, e: 0.56
episode: 7/999, score: 13, e: 0.53
episode: 8/999, score: 15, e: 0.49
episode: 9/999, score: 21, e: 0.44
episode: 10/999, score: 31, e: 0.37
episode: 11/999, score: 68, e: 0.26
episode: 12/999, score: 44, e: 0.21
episode: 13/999, score: 26, e: 0.18
episode: 14/999, score: 35, e: 0.15
episode: 15/999, score: 32, e: 0.13
episode: 16/999, score: 63, e: 0.094
episode: 17/999, score: 52, e: 0.072
episode: 18/999, score: 83, e: 0.047
episode: 19/999, score: 82, e: 0.031
episode: 20/999, score: 129, e: 0.016
episode: 21/999, score: 199, e: 0.01
episode: 22/999, score: 145, e: 0.01
episode: 23/999, score: 199, e: 0.01
episode: 24/999, score: 199, e: 0.01
episode: 25/999, score: 165, e: 0.01
episode: 26/999, score: 199, e: 0.01
episode: 27/999, score: 199

KeyboardInterrupt: 